<a href="https://colab.research.google.com/github/caocscar/workshops/blob/master/pytorch/Workshop_Regression_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Regression Problem**

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

print('Torch version', torch.__version__)
print('Pandas version', pd.__version__)
print('Numpy version', np.__version__)

Torch version 1.3.1
Pandas version 0.25.3
Numpy version 1.17.4


The following should say `cuda:0`. If it does not, we need to go to *Edit* -> *Notebook settings* and change it to a `GPU` from `None`. You only have to do this once per notebook.

In [24]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

Read in dataset

In [0]:
df_train = pd.read_csv('https://raw.githubusercontent.com/greght/Workshop-Keras-DNN/master/ChallengeProblems/dataRegression_train.csv', header=None)
df_test = pd.read_csv('https://raw.githubusercontent.com/greght/Workshop-Keras-DNN/master/ChallengeProblems/dataRegression_test.csv', header=None)

Construct our x,y variables along with the training and validation dataset

In [26]:
x_train = df_train.iloc[:,0:2]
y_train = df_train.iloc[:,2]
x_val = df_val.iloc[:,0:2]
y_val = df_val.iloc[:,2]

NameError: ignored

Preprocess our data to go from a `pandas` DataFrame to a `numpy` array to a `torch` tensor.

In [0]:
x_train_tensor = torch.tensor(x_train.to_numpy(), device=device, dtype=torch.float, requires_grad=True)
y_train_tensor = torch.tensor(y_train.to_numpy(), device=device, dtype=torch.float, requires_grad=True)
x_val_tensor = torch.tensor(x_val.to_numpy(), device=device, dtype=torch.float, requires_grad=True)
y_val_tensor = torch.tensor(y_val.to_numpy(), device=device, dtype=torch.float, requires_grad=True)
y_train_tensor = y_train_tensor.view(-1,1)
y_val_tensor = y_val_tensor.view(-1,1)

We'll write a python class to define out neural network.

In [0]:
class ThreeLayerNN(nn.Module):
    def __init__(self, dim_input, H):
        super().__init__()
        self.fc1 = nn.Linear(dim_input, H)
        self.fc2 = nn.Linear(H,H)
        self.fc3 = nn.Linear(H,1)
    
    def forward(self, x):
        x1 = F.relu(self.fc1(x))
        x2 = F.relu(self.fc2(x1))
        y_pred = self.fc3(x2)
        return y_pred

We create an instance of this class.

In [0]:
model = ThreeLayerNN(x_train_tensor.shape[1],5).to(device)
print(model)

`model.parameters()` contains the **weights** and **bias** (alternating) for each of the 3 layers



In [0]:
params = list(model.parameters())
print(f'There are {len(params)} parameters')
for param in params:
    print(param)

We'll define a template for our `fit_model` function that contains `train` and `validate` functions.

---



In [0]:
def fit_model(model, loss_fn, optimizer):
    def train(x,y):
        yhat = model(x)
        loss = loss_fn(yhat,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        return loss.item()
    
    def validate(x,y):
        yhat = model(x)
        loss = loss_fn(yhat,y)
        return loss.item()
    
    return train, validate

We define our *loss function*, *learning rate*, and our *optimizer*. We pass this to `fit_model` to return our `train` and `validate` functions.


In [0]:
loss_fn = nn.MSELoss(reduction='mean') #default
learning_rate = 0.1
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)
train, validate = fit_model(model, loss_fn, optimizer)

## Mini-batches
From the documentation: `torch.nn` only supports mini-batches. The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.

In [0]:
train_data = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=10, shuffle=True)

Here is our training loop with mini-batch processing. We have to move each mini-batch onto the GPU.

In [0]:
epochs = 100
for epoch in range(epochs):
    # training
    losses = []
    for i, (xbatch, ybatch) in enumerate(train_loader):
        xbatch = xbatch.to(device)
        ybatch = ybatch.to(device)
        loss = train(xbatch, ybatch)
        losses.append(loss)
    training_loss = np.mean(losses)
    # validation
    validation_loss = validate(x_test_tensor, y_test_tensor)
    # print intermediate results
    if epoch%10 == 9:
        print(epoch, training_loss, validation_loss)

We can view the current state of our model using the `state_dict` method.

In [0]:
model.state_dict()